In [2]:
#!/usr/bin/env python
# coding: utf-8

import os
import time
import torch
import torch.nn as nn
import torch.nn.functional as F
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, confusion_matrix
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader, random_split
from PIL import Image

from datetime import datetime
from datetime import timedelta
import csv

# --- ViT Model (No changes needed in ViT class itself, only in num_classes when instantiating) ---
from einops import rearrange, repeat
from einops.layers.torch import Rearrange

# helpers
def pair(t):
    return t if isinstance(t, tuple) else (t, t)

# classes
class FeedForward(nn.Module):
    def __init__(self, dim, hidden_dim, dropout = 0.):
        super().__init__()
        self.net = nn.Sequential(
            nn.LayerNorm(dim),
            nn.Linear(dim, hidden_dim),
            nn.GELU(),
            nn.Dropout(dropout),
            nn.Linear(hidden_dim, dim),
            nn.Dropout(dropout)
        )

    def forward(self, x):
        return self.net(x)

class Attention(nn.Module):
    def __init__(self, dim, heads = 8, dim_head = 64, dropout = 0.):
        super().__init__()
        inner_dim = dim_head * heads
        project_out = not (heads == 1 and dim_head == dim)

        self.heads = heads
        self.scale = dim_head ** -0.5

        self.norm = nn.LayerNorm(dim)

        self.attend = nn.Softmax(dim = -1)
        self.dropout = nn.Dropout(dropout)

        self.to_qkv = nn.Linear(dim, inner_dim * 3, bias = False)

        self.to_out = nn.Sequential(
            nn.Linear(inner_dim, dim),
            nn.Dropout(dropout)
        ) if project_out else nn.Identity()

    def forward(self, x):
        x = self.norm(x)

        qkv = self.to_qkv(x).chunk(3, dim = -1)
        q, k, v = map(lambda t: rearrange(t, 'b n (h d) -> b h n d', h = self.heads), qkv)

        dots = torch.matmul(q, k.transpose(-1, -2)) * self.scale

        attn = self.attend(dots)
        attn = self.dropout(attn)

        out = torch.matmul(attn, v)
        out = rearrange(out, 'b h n d -> b n (h d)')
        return self.to_out(out)

class Transformer(nn.Module):
    def __init__(self, dim, depth, heads, dim_head, mlp_dim, dropout = 0.):
        super().__init__()
        self.norm = nn.LayerNorm(dim)
        self.layers = nn.ModuleList([])
        for _ in range(depth):
            self.layers.append(nn.ModuleList([
                Attention(dim, heads = heads, dim_head = dim_head, dropout = dropout),
                FeedForward(dim, mlp_dim, dropout = dropout)
            ]))

    def forward(self, x):
        for attn, ff in self.layers:
            x = attn(x) + x
            x = ff(x) + x

        return self.norm(x)

class ViT(nn.Module):
    def __init__(self, *, image_size, patch_size, num_classes, dim, depth, heads, mlp_dim, pool = 'cls', channels = 3, dim_head = 64, dropout = 0., emb_dropout = 0.):
        super().__init__()
        image_height, image_width = pair(image_size)
        patch_height, patch_width = pair(patch_size)

        assert image_height % patch_height == 0 and image_width % patch_width == 0, 'Image dimensions must be divisible by the patch size.'

        num_patches = (image_height // patch_height) * (image_width // patch_width)
        patch_dim = channels * patch_height * patch_width
        assert pool in {'cls', 'mean'}, 'pool type must be either cls (cls token) or mean (mean pooling)'

        self.to_patch_embedding = nn.Sequential(
            Rearrange('b c (h p1) (w p2) -> b (h w) (p1 p2 c)', p1 = patch_height, p2 = patch_width),
            nn.LayerNorm(patch_dim),
            nn.Linear(patch_dim, dim),
            nn.LayerNorm(dim),
        )

        self.pos_embedding = nn.Parameter(torch.randn(1, num_patches + 1, dim))
        self.cls_token = nn.Parameter(torch.randn(1, 1, dim))
        self.dropout = nn.Dropout(emb_dropout)

        self.transformer = Transformer(dim, depth, heads, dim_head, mlp_dim, dropout)

        self.pool = pool
        self.to_latent = nn.Identity()

        # The final MLP head will output 1 logit for binary classification
        self.mlp_head = nn.Linear(dim, num_classes)

    def forward(self, img):
        x = self.to_patch_embedding(img)
        b, n, _ = x.shape

        cls_tokens = repeat(self.cls_token, '1 1 d -> b 1 d', b = b)
        x = torch.cat((cls_tokens, x), dim=1)
        x += self.pos_embedding[:, :(n + 1)]
        x = self.dropout(x)

        x = self.transformer(x)

        x = x.mean(dim = 1) if self.pool == 'mean' else x[:, 0]

        x = self.to_latent(x)
        return self.mlp_head(x)



# --- Custom Dataset (No changes needed here for binary classification specifically) ---
class CustomDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.image_paths = []
        self.labels = []
        self.classes = sorted(os.listdir(root_dir))
        self.class_to_idx = {cls_name: i for i, cls_name in enumerate(self.classes)}
        
        # Ensure there are exactly two classes for binary classification
        if len(self.classes) != 2:
            raise ValueError(f"Expected 2 classes for binary classification, but found {len(self.classes)}: {self.classes}")
        
        for cls in self.classes:
            cls_path = os.path.join(root_dir, cls)
            for img_name in os.listdir(cls_path):
                self.image_paths.append(os.path.join(cls_path, img_name))
                self.labels.append(self.class_to_idx[cls])
    
    def __len__(self):
        return len(self.image_paths)
    
    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        # Labels for BCEWithLogitsLoss need to be float and shaped [batch_size, 1]
        label = torch.tensor(self.labels[idx]).float().unsqueeze(0) 
        image = Image.open(img_path).convert('RGB')
        if self.transform:
            image = self.transform(image)
        return image, label



def evaluate_model(model, loader, criterion, device):
    model.eval()
    losses, preds, targets = [], [], []
    with torch.no_grad():
        for images, labels in loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            
            # For BCEWithLogitsLoss, labels need to be float and shape (batch_size, 1)
            # Loss calculation remains the same
            loss = criterion(outputs, labels) 
            losses.append(loss.item())
            
            # Apply sigmoid and threshold for binary predictions
            # outputs is (batch_size, 1)
            binary_preds = (torch.sigmoid(outputs) > 0.5).int().cpu().numpy()
            
            preds.extend(binary_preds.flatten()) # Flatten to 1D array
            targets.extend(labels.cpu().numpy().flatten()) # Flatten to 1D array

    acc = accuracy_score(targets, preds)
    # For binary classification, 'binary' average is often suitable for precision/recall/f1.
    # 'macro' also works if you want to treat each class equally.
    prec, recall, f1, _ = precision_recall_fscore_support(targets, preds, average='binary', zero_division=0)
    
    return np.mean(losses), acc, prec, recall, f1, preds, targets



def train_model(model, criterion, optimizer, train_loader, val_loader, test_loader, device,
                num_epochs, save_policy='min', base_output_dir='logs', class_names=None):

    # Create timestamped main directory
    now = datetime.now().strftime('%Y-%m-%d_%H-%M-%S')
   # run_dir = os.path.join(base_output_dir, f'run_{now}')
    run_dir=base_output_dir
    model_dir = os.path.join(run_dir, 'models')
    metric_dir = os.path.join(run_dir, 'metrics')
    os.makedirs(model_dir, exist_ok=True)
    os.makedirs(metric_dir, exist_ok=True)

    best_val_score = float('-inf')
    worst_val_score = float('inf') # For 'min' save policy with loss
    start_training_time = time.time()
 
    # Prepare header
    train_log_path = os.path.join(metric_dir, 'train_log.txt')
    val_log_path = os.path.join(metric_dir, 'val_log.txt')

    with open(train_log_path, 'w', newline='') as f:
        writer = csv.writer(f)
        writer.writerow(['Epoch', 'Loss', 'Accuracy', 'Precision', 'Recall', 'F1', 'Time'])

    with open(val_log_path, 'w', newline='') as f:
        writer = csv.writer(f)
        writer.writerow(['Epoch', 'Loss', 'Accuracy', 'Precision', 'Recall', 'F1', 'Time'])

    print("Starting time:", now.replace("_", " "))

    for epoch in range(1, num_epochs + 1):
        epoch_start_time = time.time()
        model.train()
        train_losses, train_preds, train_targets = [], [], []

        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            
            # Labels already formatted as float().unsqueeze(0) in CustomDataset __getitem__
            loss = criterion(outputs, labels) 
            loss.backward()
            optimizer.step()

            train_losses.append(loss.item())
            # Apply sigmoid and threshold for binary predictions
            binary_preds = (torch.sigmoid(outputs) > 0.5).int().cpu().numpy()
            train_preds.extend(binary_preds.flatten())
            train_targets.extend(labels.cpu().numpy().flatten())

        # Train metrics
        train_loss = np.mean(train_losses)
        train_acc = accuracy_score(train_targets, train_preds)
        train_prec, train_rec, train_f1, _ = precision_recall_fscore_support(train_targets, train_preds, average='binary', zero_division=0)

        # Validation metrics
        val_loss, val_acc, val_prec, val_rec, val_f1, _, _ = evaluate_model(model, val_loader, criterion, device)

        epoch_time = time.time() - epoch_start_time

        print(f"Epoch [{epoch}/{num_epochs}]")
        print(f"Train | Loss: {train_loss:.4f} | Acc: {train_acc:.4f} | Prec: {train_prec:.4f} | Rec: {train_rec:.4f} | F1: {train_f1:.4f}")
        print(f"Val   | Loss: {val_loss:.4f} | Acc: {val_acc:.4f} | Prec: {val_prec:.4f} | Rec: {val_rec:.4f} | F1: {val_f1:.4f}")
        print(f"Epoch Time: {str(timedelta(seconds=epoch_time))} sec\n")

        # Append metrics in CSV-like format to .txt files (rounded to 4 decimal places)
        with open(train_log_path, 'a', newline='') as f:
            writer = csv.writer(f)
            writer.writerow([
                epoch,
                round(train_loss, 4),
                round(train_acc, 4),
                round(train_prec, 4),
                round(train_rec, 4),
                round(train_f1, 4),
                round(epoch_time, 4)
            ])

        with open(val_log_path, 'a', newline='') as f:
            writer = csv.writer(f)
            writer.writerow([
                epoch,
                round(val_loss, 4),
                round(val_acc, 4),
                round(val_prec, 4),
                round(val_rec, 4),
                round(val_f1, 4),
                round(epoch_time, 4)
            ])

        # Save model based on policy
        epoch_model_path = os.path.join(model_dir, f'epoch_{epoch:04d}.pt')

        if save_policy == 'max':
            if val_acc > best_val_score:
                best_val_score = val_acc
                torch.save(model.state_dict(), os.path.join(model_dir, 'best_model.pt'))
                print(f"Validation accuracy improved to {val_acc:.4f}. Saving best model.")
            torch.save(model.state_dict(), epoch_model_path) # Always save current epoch model if max

        elif save_policy == 'min':
            if val_loss < worst_val_score:
                worst_val_score = val_loss
                torch.save(model.state_dict(), os.path.join(model_dir, 'best_model.pt'))
                print(f"Validation loss improved to {val_loss:.4f}. Saving best model.")
            torch.save(model.state_dict(), epoch_model_path) # Always save current epoch model if min

        elif save_policy == 'all':
            torch.save(model.state_dict(), epoch_model_path)

        elif save_policy == 'last':
            torch.save(model.state_dict(), epoch_model_path)
            # For 'last', the 'best_model.pt' will simply be the last epoch's model
            if epoch == num_epochs: 
                torch.save(model.state_dict(), os.path.join(model_dir, 'best_model.pt'))

    total_time = time.time() - start_training_time
    print("Finish time:", datetime.now().strftime("%d-%m-%Y %H:%M:%S"))
    print(f"Total training time: {str(timedelta(seconds=total_time))} seconds")

    print("\n🎯 Final Evaluation on Test Set (using last epoch's model):")
    test_loss, test_acc, test_prec, test_rec, test_f1, test_preds, test_targets = evaluate_model(model, test_loader, criterion, device)
    print(f"Test | Loss: {test_loss:.4f} | Acc: {test_acc:.4f} | Prec: {test_prec:.4f} | Rec: {test_rec:.4f} | F1: {test_f1:.4f}")

    # Confusion matrix
    cm = confusion_matrix(test_targets, test_preds)
    plt.figure(figsize=(8, 6))
    # Ensure class_names are correct for binary (e.g., ['Class 0', 'Class 1'])
    xticks = yticks = class_names if class_names and len(class_names) == 2 else ['Class 0', 'Class 1']
    sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=xticks, yticklabels=yticks)
    plt.title("Confusion Matrix")
    plt.xlabel("Predicted")
    plt.ylabel("True")
    plt.tight_layout()
    plt.savefig(os.path.join(metric_dir, "confusion_matrix.png"))
    plt.close()

   

    # Save final test metrics to a structured .txt file
    test_log_path = os.path.join(metric_dir, 'test_metrics.txt')
    with open(test_log_path, 'w', newline='') as f:
        writer = csv.writer(f)
        writer.writerow(['Loss', 'Accuracy', 'Precision', 'Recall', 'F1'])
        writer.writerow([
            round(test_loss, 4),
            round(test_acc, 4),
            round(test_prec, 4),
            round(test_rec, 4),
            round(test_f1, 4)
        ])

In [3]:
# normal train

# normal train

In [4]:
# --- Settings ---
image_dir = 'DataSet/top-agriculture-crop-disease'    # Folder where images are stored
image_size = 32 # Make sure this matches the image_size of your ViT model
batch_size = 32
val_ratio = 0.1
test_ratio = 0.1
num_workers = 4 # Adjust based on your system's capabilities
save_dir='logs'

# --- Transforms ---
train_transform = transforms.Compose([
    transforms.Resize((image_size, image_size)),
    transforms.ToTensor(),
    transforms.Normalize([0.5]*3, [0.5]*3)
])

val_test_transform = transforms.Compose([
    transforms.Resize((image_size, image_size)),
    transforms.ToTensor(),
    transforms.Normalize([0.5]*3, [0.5]*3)
])

# --- Load full dataset ---
# The CustomDataset now expects exactly 2 classes in the root_dir
full_dataset = CustomDataset(image_dir) 
num_samples = len(full_dataset)
num_val = int(num_samples * val_ratio)
num_test = int(num_samples * test_ratio)
num_train = num_samples - num_val - num_test

# Set seed for reproducibility
seed = 42  # You can change this to any integer
generator = torch.Generator().manual_seed(seed)

# --- Split dataset ---
train_dataset, val_dataset, test_dataset = random_split(full_dataset, [num_train, num_val, num_test], generator=generator)

# Assign transforms after split
# Note: For random_split, you access the original dataset's transform via .dataset
# This approach works if the transform is applied to the underlying full dataset.
# A more robust way for random_split when transforms differ is to pass the transform
# directly during the __getitem__ of the dataset wrapper, but for simplicity, this often works.
# For binary classification, make sure your data folder 'top-agriculture-crop-disease'
# contains exactly two subfolders (e.g., 'healthy' and 'diseased').
train_dataset.dataset.transform = train_transform
val_dataset.dataset.transform = val_test_transform
test_dataset.dataset.transform = val_test_transform

# --- DataLoaders ---
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=num_workers)
val_loader   = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=num_workers)
test_loader  = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=num_workers)

# --- Class Info ---
class_names = full_dataset.classes
num_classes = len(class_names) # This should now be 2
print(f"Loaded dataset with {num_classes} classes: {class_names}")


# Instantiate the ViT model for binary classification
model = ViT(
    image_size = image_size,  # Use the image_size from settings
    patch_size = 16,
    num_classes = 1,  # Set to 1 for binary classification with BCEWithLogitsLoss
    dim = 192,
    depth = 9,
    heads = 12,
    mlp_dim = 384,
    dropout = 0,
    emb_dropout = 0
)

# Use BCEWithLogitsLoss for binary classification
criterion = nn.BCEWithLogitsLoss() 
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

print(f"Using device: {device}")

# === Tarihli ana klasör oluştur ===
run_time = datetime.now().strftime('%Y-%m-%d_%H-%M-%S')
run_dir = os.path.join(save_dir, f'run_{run_time}')
os.makedirs(run_dir, exist_ok=True)
print(f"📁 Saving all folds under: {run_dir}")


# Train the model
train_model(model, criterion, optimizer, train_loader, val_loader, test_loader, device,
            num_epochs=2, save_policy='last', base_output_dir=run_dir, class_names=class_names) # Pass class_names for confusion matrix

# You can also load and evaluate a specific saved model after training if needed
# save_dir should point to the correct run directory, e.g., 'logs/run_2025-07-15_22-17-57/models'
# Replace this with the actual path of your saved model from the 'train_model' output
# save_dir_for_inference = "./logs/run_YYYY-MM-DD_HH-MM-SS/models" 
# model.load_state_dict(torch.load(os.path.join(save_dir_for_inference, 'best_model.pt')))
# _, acc, prec, rec, f1, _, _ = evaluate_model(model, test_loader, criterion, device)
# print(f"Best Model (Loaded for Inference) | Acc: {acc:.4f} | Prec: {prec:.4f} | Rec: {rec:.4f} | F1: {f1:.4f}")

Loaded dataset with 2 classes: ['scaledHealthy', 'scaledInfected']
Using device: cuda
📁 Saving all folds under: logs/run_2025-07-19_11-51-07
Starting time: 2025-07-19 11-51-07
Epoch [1/2]
Train | Loss: 0.5783 | Acc: 0.6838 | Prec: 0.6787 | Rec: 0.7534 | F1: 0.7141
Val   | Loss: 0.5010 | Acc: 0.7654 | Prec: 0.7798 | Rec: 0.7692 | F1: 0.7745
Epoch Time: 0:00:06.217449 sec

Epoch [2/2]
Train | Loss: 0.4302 | Acc: 0.8012 | Prec: 0.7889 | Rec: 0.8476 | F1: 0.8172
Val   | Loss: 0.4267 | Acc: 0.8081 | Prec: 0.8804 | Rec: 0.7330 | F1: 0.8000
Epoch Time: 0:00:05.639070 sec

Finish time: 19-07-2025 11:51:20
Total training time: 0:00:12.355932 seconds

🎯 Final Evaluation on Test Set (using last epoch's model):
Test | Loss: 0.4013 | Acc: 0.8009 | Prec: 0.9022 | Rec: 0.7155 | F1: 0.7981


# Kfold

In [ ]:
from sklearn.model_selection import StratifiedKFold
from torch.utils.data import Subset
import numpy as np
import torch
import os

from datetime import datetime

# === Tarihli ana klasör oluştur ===
run_time = datetime.now().strftime('%Y-%m-%d_%H-%M-%S')
run_dir = os.path.join(save_dir, f'run_{run_time}')
os.makedirs(run_dir, exist_ok=True)
print(f"📁 Saving all folds under: {run_dir}")

# === Parameters ===
k_folds = 5
image_dir = 'DataSet/top-agriculture-crop-disease'
image_size = 256
batch_size = 32
num_epochs = 1
save_policy = 'last'
save_dir = 'logs'
test_ratio = 0.1
num_workers = 4
seed = 42

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# === Transforms ===
train_transform = transforms.Compose([
    transforms.Resize((image_size, image_size)),
    transforms.ToTensor(),
    transforms.Normalize([0.5]*3, [0.5]*3)
])
val_test_transform = transforms.Compose([
    transforms.Resize((image_size, image_size)),
    transforms.ToTensor(),
    transforms.Normalize([0.5]*3, [0.5]*3)
])

# === Load full dataset ===
full_dataset = CustomDataset(image_dir)
class_names = full_dataset.classes
num_classes = len(class_names)

# === Step 1: Split off test set ===
total_size = len(full_dataset)
test_size = int(total_size * test_ratio)
trainval_size = total_size - test_size

generator = torch.Generator().manual_seed(seed)
trainval_dataset, test_dataset = random_split(full_dataset, [trainval_size, test_size], generator=generator)

# Fix transforms
trainval_dataset.dataset.transform = None  # Will apply inside each fold
test_dataset.dataset.transform = val_test_transform

# StratifiedKFold için: trainval_dataset içindeki sıralamaya göre alınır
X = list(range(len(trainval_dataset)))
y = [int(label.item()) for _, label in trainval_dataset]

skf = StratifiedKFold(n_splits=k_folds, shuffle=True, random_state=seed)

all_fold_results = []

for fold, (train_idx, val_idx) in enumerate(skf.split(X, y)):
    print(f"\n🌀 Fold {fold+1}/{k_folds}")

    train_subset = Subset(trainval_dataset, train_idx)
    val_subset = Subset(trainval_dataset, val_idx)

    # Transforms uygulanır
    train_subset.dataset.transform = train_transform
    val_subset.dataset.transform = val_test_transform

    # DataLoader'lar
    train_loader = DataLoader(train_subset, batch_size=batch_size, shuffle=True, num_workers=num_workers)
    val_loader = DataLoader(val_subset, batch_size=batch_size, shuffle=False, num_workers=num_workers)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=num_workers)

    # Model, optimizer, criterion yeniden tanımlanır
    model = ViT(
        image_size=image_size,
        patch_size=16,
        num_classes=1,
        dim=192,
        depth=9,
        heads=12,
        mlp_dim=384,
        dropout=0,
        emb_dropout=0
    ).to(device)

    criterion = nn.BCEWithLogitsLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

    
     # Fold için klasör oluştur
    fold_save_dir = os.path.join(run_dir, f'fold_{fold+1}')
    os.makedirs(fold_save_dir, exist_ok=True)

    train_model(model, criterion, optimizer, train_loader, val_loader, test_loader, device,
                num_epochs=num_epochs, save_policy=save_policy,
                base_output_dir=fold_save_dir, class_names=class_names)

     # Final test evaluation
    test_loss, test_acc, test_prec, test_rec, test_f1, _, _ = evaluate_model(model, test_loader, criterion, device)

    with open(os.path.join(fold_save_dir, 'test_metrics.txt'), 'w', newline='') as f:
        writer = csv.writer(f)
        writer.writerow(['Loss', 'Accuracy', 'Precision', 'Recall', 'F1'])
        writer.writerow([
            round(test_loss, 4),
            round(test_acc, 4),
            round(test_prec, 4),
            round(test_rec, 4),
            round(test_f1, 4)
        ])

    all_fold_results.append({
        'fold': fold + 1,
        'loss': test_loss,
        'acc': test_acc,
        'prec': test_prec,
        'rec': test_rec,
        'f1': test_f1
    })

# ==== Save average results as CSV-formatted text ====
avg_metrics = {
    'loss': np.mean([r['loss'] for r in all_fold_results]),
    'acc': np.mean([r['acc'] for r in all_fold_results]),
    'prec': np.mean([r['prec'] for r in all_fold_results]),
    'rec': np.mean([r['rec'] for r in all_fold_results]),
    'f1': np.mean([r['f1'] for r in all_fold_results])
}

with open(os.path.join(run_dir, 'kfold_results.txt'), 'w', newline='') as f:
    writer = csv.writer(f)
    writer.writerow(['Fold', 'Loss', 'Accuracy', 'Precision', 'Recall', 'F1'])
    for r in all_fold_results:
        writer.writerow([
            r['fold'],
            round(r['loss'], 4),
            round(r['acc'], 4),
            round(r['prec'], 4),
            round(r['rec'], 4),
            round(r['f1'], 4)
        ])
    writer.writerow([])
    writer.writerow(['Average', 
                     round(avg_metrics['loss'], 4),
                     round(avg_metrics['acc'], 4),
                     round(avg_metrics['prec'], 4),
                     round(avg_metrics['rec'], 4),
                     round(avg_metrics['f1'], 4)])


📁 Saving all folds under: logs/run_2025-07-19_11-52-15
Using device: cuda

🌀 Fold 1/5
Starting time: 2025-07-19 11-52-53
Epoch [1/2]
Train | Loss: 0.5017 | Acc: 0.7538 | Prec: 0.7295 | Rec: 0.8425 | F1: 0.7820
Val   | Loss: 0.4030 | Acc: 0.8187 | Prec: 0.8655 | Rec: 0.7744 | F1: 0.8175
Epoch Time: 0:00:20.334889 sec

Epoch [2/2]
Train | Loss: 0.3944 | Acc: 0.8281 | Prec: 0.8271 | Rec: 0.8494 | F1: 0.8381
Val   | Loss: 0.3199 | Acc: 0.8660 | Prec: 0.8293 | Rec: 0.9373 | F1: 0.8800
Epoch Time: 0:00:19.951195 sec

Finish time: 19-07-2025 11:53:34
Total training time: 0:00:40.873141 seconds

🎯 Final Evaluation on Test Set (using last epoch's model):
Test | Loss: 0.3329 | Acc: 0.8555 | Prec: 0.8406 | Rec: 0.9095 | F1: 0.8737

🌀 Fold 2/5
Starting time: 2025-07-19 11-53-37
Epoch [1/2]
Train | Loss: 0.5330 | Acc: 0.7337 | Prec: 0.7248 | Rec: 0.7930 | F1: 0.7573
Val   | Loss: 0.3551 | Acc: 0.8371 | Prec: 0.8412 | Rec: 0.8496 | F1: 0.8454
Epoch Time: 0:00:20.024232 sec

Epoch [2/2]
Train | Loss:

# load model

In [ ]:
save_dir="./runs/run_2025-07-15_22-17-57/models"

model.load_state_dict(torch.load(os.path.join(save_dir, 'best_model.pt')))
_, acc, prec, rec, f1, _, _ = evaluate_model(model, test_loader, criterion, device)
print(f"Best Model | Acc: {acc:.4f} | Prec: {prec:.4f} | Rec: {rec:.4f} | F1: {f1:.4f}")